# Příprava datasetu


In [1]:
import pandas as pd 
import numpy as np 
from sklearn.model_selection import StratifiedKFold, cross_validate, cross_val_predict 
from sklearn.naive_bayes import GaussianNB 
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import confusion_matrix, f1_score, make_scorer

file_path = "weatherAUS.csv"
df = pd.read_csv(file_path)

n_records_before = len(df)
print(f"Celkový počet záznamů: {n_records_before}")

location_counts = df['Location'].value_counts()
most_data_location = location_counts.index[0]
print(f"Město s největším počtem dat: {most_data_location} ({location_counts.max()} záznamů)")

df_filtered = df[df['Location'] == most_data_location].copy()
n_records_location = len(df_filtered)
print(f"Počet záznamů v {most_data_location} před filtrováním: {n_records_location}")

target_column = 'RainTomorrow'

df_filtered = df_filtered.drop(columns=['RISK_MM', 'Location', 'Date'])
df_filtered[target_column] = df_filtered[target_column].map({'No': 0, 'Yes': 1})
df_filtered.dropna(subset=[target_column], inplace=True)

threshold = 0.6
missing_ratio = df_filtered.isnull().mean()
columns_to_drop = missing_ratio[missing_ratio > threshold].index.tolist()
print(f"Odstraňuji sloupce s více než {threshold * 100:.0f}% NA: {columns_to_drop}")
df_filtered.drop(columns=columns_to_drop, inplace=True)

wind_direction_cols = ['WindGustDir', 'WindDir9am', 'WindDir3pm']
direction_map = {
    'N': 0, 'NNE': 22.5, 'NE': 45, 'ENE': 67.5, 'E': 90, 'ESE': 112.5,
    'SE': 135, 'SSE': 157.5, 'S': 180, 'SSW': 202.5, 'SW': 225, 'WSW': 247.5,
    'W': 270, 'WNW': 292.5, 'NW': 315, 'NNW': 337.5
}

for col in wind_direction_cols:
    if col in df_filtered.columns:
        df_filtered[col + '_angle'] = df_filtered[col].map(direction_map)
        angles_rad = np.deg2rad(df_filtered[col + '_angle'])
        df_filtered[col + '_sin'] = np.sin(angles_rad)
        df_filtered[col + '_cos'] = np.cos(angles_rad)
        df_filtered = df_filtered.drop(columns=[col, col + '_angle'])

if 'RainToday' in df_filtered.columns:
    df_filtered['RainToday'] = df_filtered['RainToday'].map({'No': 0, 'Yes': 1})

df_clean = df_filtered.dropna()

n_records_after = len(df_clean)
print(f"Počet záznamů v {most_data_location} po filtrování a čištění NA: {n_records_after}")

X = df_clean.drop(columns=[target_column])
y = df_clean[target_column]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns, index=X.index)

N_SPLITS = 10
skf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
scoring = ['accuracy', 'precision', 'recall', 'f1']


Celkový počet záznamů: 142193
Město s největším počtem dat: Canberra (3418 záznamů)
Počet záznamů v Canberra před filtrováním: 3418
Odstraňuji sloupce s více než 60% NA: []
Počet záznamů v Canberra po filtrování a čištění NA: 1078


#  Gaussův naivní Bayesův klasifikátor (Gaussian naive Bayes classifier)


In [2]:
gnb = GaussianNB()

gnb_results = cross_validate(gnb, X_scaled_df, y, cv=skf, scoring=scoring, return_train_score=False)

print("\nPrůměrné a nejlepší metriky pro GNB (napříč 10 foldy):")
for metric in scoring:
    scores = gnb_results[f'test_{metric}']
    print(f"  {metric.capitalize():<10}: Průměr: {np.mean(scores):.4f} | Nejlepší: {np.max(scores):.4f}")

y_pred_gnb = cross_val_predict(gnb, X_scaled_df, y, cv=skf)
cm_gnb = confusion_matrix(y, y_pred_gnb)

print("\nKontingenční tabulka (GNB) - Agregovaná pro všechny foldy:")
print(pd.DataFrame(cm_gnb, 
                   index=['Skutečné NE (0)', 'Skutečné ANO (1)'], 
                   columns=['Predikované NE (0)', 'Predikované ANO (1)']))



Průměrné a nejlepší metriky pro GNB (napříč 10 foldy):
  Accuracy  : Průměr: 0.8256 | Nejlepší: 0.8692
  Precision : Průměr: 0.5634 | Nejlepší: 0.6667
  Recall    : Průměr: 0.6433 | Nejlepší: 0.7727
  F1        : Průměr: 0.5981 | Nejlepší: 0.6957

Kontingenční tabulka (GNB) - Agregovaná pro všechny foldy:
                  Predikované NE (0)  Predikované ANO (1)
Skutečné NE (0)                  749                  110
Skutečné ANO (1)                  78                  141


# k-nejbližší sousedé (k-Nearest Neighbours)


In [3]:
k_values = [i for i in range(1, 101) if i % 2 != 0]

best_f1_score = 0.0
best_k = None

for k in k_values:
    knn = KNeighborsClassifier(n_neighbors=k)
    f1_scores = cross_validate(knn, X_scaled_df, y, cv=skf, scoring={'f1': make_scorer(f1_score)})['test_f1']
    mean_f1 = np.mean(f1_scores)
    if mean_f1 > best_f1_score:
        best_f1_score = mean_f1
        best_k = k

knn_best = KNeighborsClassifier(n_neighbors=best_k)
knn_results = cross_validate(knn_best, X_scaled_df, y, cv=skf, scoring=scoring)

print(f"\nPrůměrné a nejlepší metriky pro kNN (napříč 10 foldy a pro optimální K = {best_k}):")
for metric in scoring:
    scores = knn_results[f'test_{metric}']
    print(f"  {metric.capitalize():<10}: Průměr: {np.mean(scores):.4f} | Nejlepší: {np.max(scores):.4f}")

y_pred_knn = cross_val_predict(knn_best, X_scaled_df, y, cv=skf)
cm_knn = confusion_matrix(y, y_pred_knn)

print("\nKontingenční tabulka (kNN) - Agregovaná pro všechny foldy:")
print(pd.DataFrame(cm_knn, 
                   index=['Skutečné NE (0)', 'Skutečné ANO (1)'], 
                   columns=['Predikované NE (0)', 'Predikované ANO (1)']))



Průměrné a nejlepší metriky pro kNN (napříč 10 foldy a pro optimální K = 3):
  Accuracy  : Průměr: 0.8386 | Nejlepší: 0.8796
  Precision : Průměr: 0.6407 | Nejlepší: 0.8000
  Recall    : Průměr: 0.4381 | Nejlepší: 0.6818
  F1        : Průměr: 0.5145 | Nejlepší: 0.6818

Kontingenční tabulka (kNN) - Agregovaná pro všechny foldy:
                  Predikované NE (0)  Predikované ANO (1)
Skutečné NE (0)                  808                   51
Skutečné ANO (1)                 123                   96


# Zajímavé poznatky


## Nejlepší výsledek bez ohledu na průměrné výsledky

V případě nastavení "threshold = 0.4" pro zahození sloupců s počtem řádků s NA vyšší než 40% celkového počtu záznamů dochází k zahození sloupců ['Evaporation', 'Sunshine'].
Současně ale také dochází k velkému zlepšení výsledků v nejlepším průchodu GNB:

### GNB

Průměrné a nejlepší metriky pro GNB (napříč 10 foldy):

Accuracy  : Průměr: 0.7934 | Nejlepší: 0.8889

Precision : Průměr: 0.5747 | Nejlepší: 0.7895

Recall    : Průměr: 0.6065 | Nejlepší: 0.7500

F1        : Průměr: 0.5882 | Nejlepší: 0.7692

Výsledky nejlepšího průchodu jsou však vyváženy horšími průměrnými výsledky.


## Nejlepší průměrné výsledky

Nejlepšími průměrnými výsledky napříč průchody proto zůstává nastavení "threshold = 0.6" a výše, které sice nezahazuje žádné sloupce, ale průměrné hodnocení obou algoritmů dosahuje nejvyšších hodnot napříč různými konfiguracemi. Dle mého měření ani při použití pouze 1078 záznamů z celkových 3418 nedošlo k overfittingu.

### GNB:

Průměrné a nejlepší metriky pro GNB (napříč 10 foldy):

Accuracy  : Průměr: 0.8256 | Nejlepší: 0.8692

Precision : Průměr: 0.5634 | Nejlepší: 0.6667

Recall    : Průměr: 0.6433 | Nejlepší: 0.7727

F1        : Průměr: 0.5981 | Nejlepší: 0.6957

### kNN:

Průměrné a nejlepší metriky pro kNN (napříč 10 foldy a pro optimální K = 3):

Accuracy  : Průměr: 0.8386 | Nejlepší: 0.8796

Precision : Průměr: 0.6407 | Nejlepší: 0.8000

Recall    : Průměr: 0.4381 | Nejlepší: 0.6818

F1        : Průměr: 0.5145 | Nejlepší: 0.6818
